# Euler 7

https://projecteuler.net/problem=7

> By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13.

> What is the 10 001st prime number?

I guess the fastest solution is by using a sieve.

*    Find a number which, for sure, is greater then the nth prime
*https://en.wikipedia.org/wiki/Prime-counting_function#Inequalities*

In [17]:
function get_upper_bound(nth_prime)
    ub::Integer = ceil(nth_prime * log(nth_prime * log(nth_prime)))
    max(ub, 7)
end

get_upper_bound (generic function with 1 method)

Julia has a dynamic type system, but we can add [type annotation][type_doc] to variables, like *ub*, to 
generate more efficient code.

Moreover, `get_upper_bound` is a *generic function*. We can write several implementations of it, by adding different
type specialization of its arguments ([*multimethods*][methods]). Then, Julia chooses which version to run by implementing *multiple dispatch*.

In [26]:
target_nth_prime = 10001
ub = get_upper_bound(target_nth_prime)

114320

*    Build an array of booleans $v$, s.t. $v[i]$ is false if $i$ is prime.

Julia has a very convenient bit array for this purpose. **Julia arrays are indexed starting from 1, not 0.**

In [34]:
v = trues(ub)
v[0]

LoadError: BoundsError()
while loading In[34], in expression starting on line 2

*    run the sieve and get the nth prime

First of all I try the [Sieve of Eratosthenes][eratosthenes].

In [35]:
function eratosthenes_sieve(v, target_nth)
    n = size(v, 1)
    v[1] = false
    for i = 2:n
        if v[i]
            # this is faster than v[i^2:i:n] = false
            for j=i^2:i:n
                v[j] = false
            end
            target_nth -= 1
            if 0 == target_nth
                return i
            end
        end
    end
end

@time eratosthenes_sieve(v, target_nth_prime)

elapsed time: 0.008924483 seconds (221936 bytes allocated)


104743

The [sieve of Sundaram][sundaram] is slightly more complex. Compared to Eratosthenes's, it skips even numbers.


In [36]:
function sundaram_sieve(v, target_nth)
    n = div(size(v, 1), 2)
    target_nth -= 1 # it skips 2
    for i = 1:n
        step = i * 2 + 1
        for j=i+i*step:step:n
            v[j] = false
        end
        if v[i]
            target_nth -= 1
            if 0 == target_nth
                return i*2+1
            end
        end
    end
end

v = trues(ub)
@time sundaram_sieve(v, target_nth_prime)

elapsed time: 0.008864525 seconds (264072 bytes allocated)


104743

[type_doc]: http://julia.readthedocs.org/en/latest/manual/types/
[methods]: http://julia.readthedocs.org/en/latest/manual/methods/#man-methods
[eratosthenes]: https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes
[sundaram]: https://en.wikipedia.org/wiki/Sieve_of_Sundaram